# Clustering Crypto

In [68]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [69]:
# Load the crypto_data.csv dataset.

file_path = "crypto_data.csv"
df_crypto = pd.read_csv(file_path, index_col=0)

df_crypto.head(10)

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,3.14159E+11
2015,2015 coin,X11,True,PoW/PoS,NaN,0
BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


In [70]:
# Keep all the cryptocurrencies that are being traded.
df_crypto = df_crypto[df_crypto['IsTrading'] == True]
print(df_crypto.head())
print(df_crypto.shape)

      CoinName Algorithm  IsTrading ProofType  TotalCoinsMined TotalCoinSupply
42     42 Coin    Scrypt       True   PoW/PoS     4.199995e+01              42
365    365Coin       X11       True   PoW/PoS              NaN      2300000000
404    404Coin    Scrypt       True   PoW/PoS     1.055185e+09       532000000
611  SixEleven   SHA-256       True       PoW              NaN          611000
808        808   SHA-256       True   PoW/PoS     0.000000e+00               0
(1144, 6)


In [71]:
# Keep all the cryptocurrencies that have a working algorithm.
df_crypto = df_crypto[df_crypto['Algorithm'].notna()]
df_crypto.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [72]:
# Remove the "IsTrading" column. 
df_crypto.drop(columns=['IsTrading'], inplace=True)
df_crypto.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [73]:
# Remove rows that have at least 1 null value.
df_crypto = df_crypto.dropna()
print(df_crypto.shape)
df_crypto.head(10)

(685, 5)


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,3.14159E+11
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,PoW,6.303924e+07,84000000
DASH,Dash,X11,PoW/PoS,9.031294e+06,22000000
XMR,Monero,CryptoNight-V7,PoW,1.720114e+07,0
ETC,Ethereum Classic,Ethash,PoW,1.133597e+08,210000000


In [74]:
# Keep the rows where coins are mined.
df_crypto = df_crypto[df_crypto['TotalCoinsMined'] != 0]
df_crypto.head(10)

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,3.14159E+11
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,PoW,6.303924e+07,84000000
DASH,Dash,X11,PoW/PoS,9.031294e+06,22000000
XMR,Monero,CryptoNight-V7,PoW,1.720114e+07,0
ETC,Ethereum Classic,Ethash,PoW,1.133597e+08,210000000
ZEC,ZCash,Equihash,PoW,7.383056e+06,21000000


In [75]:
# Create a new DataFrame that holds only the cryptocurrencies names.
df_names = pd.DataFrame(df_crypto['CoinName'])
crypto_index = df_names.index.tolist()
df_names.head(10)


,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum
LTC,Litecoin
DASH,Dash
XMR,Monero
ETC,Ethereum Classic
ZEC,ZCash


In [76]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
df_crypto.drop(columns=['CoinName'], inplace=True)
df_crypto.head(10)

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,3.14159E+11
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0
LTC,Scrypt,PoW,6.303924e+07,84000000
DASH,X11,PoW/PoS,9.031294e+06,22000000
XMR,CryptoNight-V7,PoW,1.720114e+07,0
ETC,Ethash,PoW,1.133597e+08,210000000
ZEC,Equihash,PoW,7.383056e+06,21000000


In [77]:
# Use get_dummies() to create variables for text features.
df_crypto_dummy = pd.get_dummies(df_crypto, columns = ['Algorithm', 'ProofType'])
print(df_crypto_dummy.shape)
df_crypto_dummy.head(10)

(533, 100)


,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,3.14159E+11,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
LTC,6.303924e+07,84000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DASH,9.031294e+06,22000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
XMR,1.720114e+07,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETC,1.133597e+08,210000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ZEC,7.383056e+06,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [78]:
# Standardize the data with StandardScaler().
crypto_scaled = StandardScaler().fit_transform(df_crypto_dummy)
print(crypto_scaled[0:5])

[[-0.11674787 -0.15286467 -0.0433555  -0.0433555  -0.0433555  -0.06137164
  -0.07523548 -0.0433555  -0.06137164 -0.06137164 -0.0433555  -0.0433555
  -0.19226279 -0.06137164 -0.09731237 -0.0433555  -0.11536024 -0.07523548
  -0.0433555  -0.0433555  -0.15176505 -0.0433555  -0.13105561 -0.0433555
  -0.0433555  -0.08695652 -0.0433555  -0.0433555  -0.0433555  -0.0433555
  -0.06137164 -0.0433555  -0.08695652 -0.08695652 -0.08695652 -0.0433555
  -0.13105561 -0.13827675 -0.13827675 -0.0433555  -0.06137164 -0.0433555
  -0.07523548 -0.1815096  -0.0433555  -0.0433555  -0.0433555  -0.07523548
  -0.15811388 -0.3145935  -0.0433555  -0.08695652 -0.07523548 -0.06137164
  -0.0433555   1.38873015 -0.0433555  -0.0433555  -0.06137164 -0.0433555
  -0.0433555  -0.0433555  -0.0433555  -0.0433555  -0.0433555  -0.0433555
  -0.0433555  -0.39836623 -0.0433555  -0.1815096  -0.0433555  -0.08695652
  -0.08695652 -0.10670145 -0.0433555  -0.0433555  -0.13105561 -0.0433555
  -0.0433555  -0.0433555  -0.0433555  -0.07523

### Deliverable 2: Reducing Data Dimensions Using PCA

In [79]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(crypto_scaled)
crypto_pca

array([[-0.33614353,  1.03632686, -0.57550558],
       [-0.31950104,  1.03659543, -0.57562851],
       [ 2.30738538,  1.68079783, -0.64162463],
       ...,
       [ 0.32622275, -2.28324039,  0.41157062],
       [-0.13967517, -2.0628675 ,  0.44155096],
       [-0.29061718,  0.81735508, -0.20362732]])

In [80]:
# Create a DataFrame with the three principal components.
df_crypto_pca = pd.DataFrame(data=crypto_pca, index=crypto_index, columns=['PC 1', 'PC 2', 'PC 3'])
df_crypto_pca.head()

,PC 1,PC 2,PC 3
42,-0.336144,1.036327,-0.575506
404,-0.319501,1.036595,-0.575629
1337,2.307385,1.680798,-0.641625
BTC,-0.142236,-1.337734,0.223717
ETH,-0.147918,-2.048636,0.413564


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [81]:
# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1, 11))

for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(df_crypto_pca)
    inertia.append(km.inertia_)

elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)

C:\Users\rjanke\Anaconda3\envs\mlenv\lib\site-packages\sklearn\cluster\_kmeans.py:1039: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  "KMeans is known to have a memory leak on Windows "


:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [90]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(df_crypto_pca)

# Predict clusters
predictions = model.predict(df_crypto_pca)

class_df = pd.DataFrame(predictions, index=crypto_index)
class_df = class_df.rename(columns={0: 'class'})
class_df

,class
42,0
404,0
1337,0
BTC,3
ETH,3
...,...
ZEPH,0
GAP,0
BDX,3
ZEN,3


In [91]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
new_crypto_df = pd.concat([df_crypto, df_crypto_pca], axis=1)

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
new_crypto_coin_df = pd.concat([new_crypto_df, df_names], axis=1)
new_crypto_coin_df.head()
#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df = pd.concat([new_crypto_coin_df, class_df], axis=1)

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(533, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.336144,1.036327,-0.575506,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.319501,1.036595,-0.575629,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,3.14159E+11,2.307385,1.680798,-0.641625,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.142236,-1.337734,0.223717,Bitcoin,3
ETH,Ethash,PoW,1.076842e+08,0,-0.147918,-2.048636,0.413564,Ethereum,3
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.166398,-1.107154,0.020101,Litecoin,3
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.397915,1.217390,-0.580449,Dash,0
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.149273,-2.248512,0.386156,Monero,3
ETC,Ethash,PoW,1.133597e+08,210000000,-0.146362,-2.048723,0.413569,Ethereum Classic,3
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.139674,-2.062867,0.441551,ZCash,3


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [99]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x='PC 1',
    y='PC 2',
    z='PC 3',
    color='class',
    symbol='class',
    width=800,
    hover_name=clustered_df['CoinName'],
    hover_data=['Algorithm']
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

In [103]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table(columns=['CoinName', 'Algorithm', 'ProofType', 'TotalCoinSupply', 'TotalCoinsMined', 'class'], sortable=True, selectable=True)

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,class]

In [106]:
# Print the total number of tradable cryptocurrencies.
count = clustered_df['CoinName'].count()
print(f'There are {count} tradable cryptocurrencies.')

There are 533 tradable cryptocurrencies.


In [111]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
X = clustered_df.drop(['Algorithm', 'ProofType', 'PC 1', 'PC 2', 'PC 3', 'CoinName', 'class'], axis=1)
X
scaled = MinMaxScaler().fit_transform(X)

In [112]:
scaled

array([[5.94229956e-03, 4.20000000e-11],
       [7.00182106e-03, 5.32000000e-04],
       [3.53420580e-02, 3.14159000e-01],
       ...,
       [6.92655067e-03, 1.40022261e-03],
       [5.94962604e-03, 2.10000000e-05],
       [5.94242837e-03, 1.00000000e-06]])

In [117]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
plot_df = pd.DataFrame(scaled, index=crypto_index, columns=['TotalCoinSupply', 'TotalCoinsMined'])

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
plot_df['CoinName'] = clustered_df['CoinName']

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
plot_df['class'] = clustered_df['class']

plot_df.head(10)

,TotalCoinSupply,TotalCoinsMined,CoinName,class
42,0.005942,4.200000e-11,42 Coin,0
404,0.007002,5.320000e-04,404Coin,0
1337,0.035342,3.141590e-01,EliteCoin,0
BTC,0.005960,2.100000e-05,Bitcoin,3
ETH,0.006050,0.000000e+00,Ethereum,3
LTC,0.006006,8.400000e-05,Litecoin,3
DASH,0.005951,2.200000e-05,Dash,0
XMR,0.005960,0.000000e+00,Monero,3
ETC,0.006056,2.100000e-04,Ethereum Classic,3
ZEC,0.005950,2.100000e-05,ZCash,3


In [118]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
plot_df.hvplot.scatter(
    x="TotalCoinsMined",
    y="TotalCoinSupply",
    hover_cols=["CoinName"],
    by="class",
)


:NdOverlay   [class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)